In [1]:
import pandas as pd
import plotly.graph_objects as go


In [29]:
df_ex = pd.read_csv('data_cleaned/bot_ex.csv', parse_dates=['date']).set_index('date')
df_im = pd.read_csv('data_cleaned/bot_im.csv', parse_dates=['date']).set_index('date')


In [30]:
df_ex

,Exports,Food,Beverages and tobacco,Crude materials,Mineral fuel and lubricant,Animal and vegetable oils and fats,Chemicals,Manufactured goods,Machinery,Miscellaneous manufactured goods,Miscellaneous transactions and commodities,Re-exports 1/
date,,,,,,,,,,,,
1995-01-31,99967.66,20761.53,267.20,7115.78,643.21,70.07,3499.40,11425.35,29661.84,25098.28,1138.73,286.27
1995-02-28,99718.05,20045.32,285.84,6764.47,1267.39,84.74,4102.98,10950.71,31484.79,23733.64,878.39,119.76
1995-03-31,128892.61,26498.59,392.87,7700.59,1310.48,85.44,4762.98,15884.52,40543.63,30056.71,1501.91,154.88
1995-04-30,100018.79,21696.83,285.28,6084.33,994.51,25.63,4252.56,11190.23,32110.82,22280.98,922.80,174.82
1995-05-31,119277.38,27013.30,275.78,7179.50,725.36,33.24,6361.54,13115.51,38287.04,24791.06,1337.55,157.51
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,834018.29,142295.62,6625.14,40299.83,29327.53,6586.57,72491.39,96929.73,367138.24,59622.37,10428.02,2273.85
2024-05-31,960220.44,155163.38,7829.11,45974.75,32078.86,7146.63,83016.93,114749.90,420899.84,71004.57,21500.58,855.88
2024-06-30,892766.41,125887.99,6528.57,38946.34,33128.18,4803.97,75025.30,103751.69,419144.20,65448.16,19735.93,366.08


In [120]:
def create_value_chart(df, li_columns, str_agg_column):
    # Create the figure and add the components
    n = 12
    df = df.tail(n=n)
    idx = df.index
    fig = go.Figure(
        data=[
            go.Bar(
                name=li_columns[0], 
                x=idx, 
                y=df[li_columns[0]],
                hovertemplate='%{y:,.0f} THB Million'
            ), 
            go.Bar(
                name=li_columns[1], 
                x=idx, 
                y=df[li_columns[1]],
                hovertemplate='%{y:,.0f} THB Million'
            ), 
            go.Bar(
                name=li_columns[2], 
                x=idx, 
                y=df[li_columns[2]],
                hovertemplate='%{y:,.0f} THB Million'
            ), 
            go.Bar(
                name=li_columns[3], 
                x=idx, 
                y=df[li_columns[3]],
                hovertemplate='%{y:,.0f} THB Million'
            ), 
            go.Bar(
                name='Others', 
                x=idx, 
                y=df[str_agg_column] - df[li_columns[0]] - df[li_columns[1]] - df[li_columns[2]] - df[li_columns[3]],
                hovertemplate='%{y:,.0f} THB Million'
            ), 
            go.Scatter(
                name=str_agg_column, 
                x=idx, 
                y=df[str_agg_column], 
                mode='lines+markers', 
                line=dict(color='black'),
                hovertemplate='%{y:,.0f} THB Million',
            )
        ]
    )

    # Customize the layout to stack the bars
    fig.update_layout(
        barmode='stack',
        bargap=0,
        bargroupgap=0,
        legend_title='Legend',
        xaxis=dict(
            tickformat='%b %y',
            tickangle=-90,
            tickmode='array',
            tickvals=df.index,
        )
    )
    
    return fig

In [121]:
fig = create_value_chart(
    df=df_ex, 
    li_columns=['Machinery', 'Food', 'Manufactured goods', 'Chemicals'], 
    str_agg_column='Exports'
)
fig.update_layout(
    title='Exports in THB Million',
)
fig.show()

In [122]:
fig = create_value_chart(
    df=df_im, 
    li_columns=['Machinery', 'Food', 'Manufactured goods', 'Chemicals'], 
    str_agg_column='Imports'
)
fig.update_layout(
    title='Imports in THB Million',
)
fig.show()

In [123]:
def create_ctg_chart(df, li_columns, str_agg_column):
    df = (df - df.shift(12)).div(df[str_agg_column], axis=0)
    # Create the figure and add the components
    n = 12
    df = df.tail(n)
    idx = df.index
    fig = go.Figure(
        data=[
            go.Bar(
                name=li_columns[0], 
                x=idx, 
                y=df[li_columns[0]],
                hovertemplate='%{y:.1%}',
            ), 
            go.Bar(
                name=li_columns[1], 
                x=idx, 
                y=df[li_columns[1]],
                hovertemplate='%{y:.1%}',
            ), 
            go.Bar(
                name=li_columns[2], 
                x=idx, 
                y=df[li_columns[2]],
                hovertemplate='%{y:.1%}',
            ), 
            go.Bar(
                name=li_columns[3], 
                x=idx, 
                y=df[li_columns[3]],
                hovertemplate='%{y:.1%}',
            ), 
            go.Bar(
                name='Others', 
                x=idx, 
                y=df[str_agg_column] - df[li_columns[0]] - df[li_columns[1]] - df[li_columns[2]] - df[li_columns[3]],
                hovertemplate='%{y:.1%}',
            ), 
            go.Scatter(
                name=str_agg_column, 
                x=idx, 
                y=df[str_agg_column], 
                mode='lines+markers', 
                line=dict(color='black'),
                hovertemplate='%{y:.1%}',
            )
        ]
    )

    # Customize the layout to stack the bars
    fig.update_layout(
        barmode='relative',
        bargap=0,
        bargroupgap=0,
        legend_title='Legend',
        xaxis=dict(
            tickformat='%b %y',
            tickangle=-90,
            tickmode='array',
            tickvals=df.index,
        ),
        yaxis=dict(
            tickformat='.1%', 
        ),
    )
    
    return fig

In [124]:
fig = create_ctg_chart(
    df=df_ex, 
    li_columns=['Machinery', 'Food', 'Manufactured goods', 'Chemicals'], 
    str_agg_column='Exports'
)
fig.update_layout(
    title='Exports Contribution to Growth (%YoY)',
)
fig.show()

In [125]:
fig = create_ctg_chart(
    df=df_im, 
    li_columns=['Machinery', 'Food', 'Manufactured goods', 'Chemicals'], 
    str_agg_column='Imports'
)
fig.update_layout(
    title='Imports Contribution to Growth (%YoY)',
)
fig.show()

In [89]:
(df_im - df_im.shift(12)).div(df_im.Imports, axis=0)

,Imports,Food,Beverages and tobacco,Crude materials,Mineral fuel and lubricant,Animal and vegetable oils and fats,Chemicals,Manufactured goods,Machinery,Miscellaneous manufactured goods,Miscellaneous transactions and commodities,Gold
date,,,,,,,,,,,,
1995-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,0.125170,0.000535,-0.000537,0.003206,-0.023851,0.000120,0.011936,0.013672,0.063252,0.008426,0.000033,0.048378
2024-05-31,0.052943,-0.005845,-0.000322,-0.001456,0.015271,-0.000063,0.003161,-0.000809,0.028661,0.007130,0.000015,0.007200
2024-06-30,0.052931,-0.003580,-0.000515,0.001426,0.002230,0.000129,-0.001470,0.005965,0.020656,0.005482,0.000042,0.022567
